In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from catboost import Pool,CatBoostClassifier
from sklearn.metrics import roc_auc_score as AUC
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv', index_col=0)
bulk = pd.read_csv('machine failure.csv')
bulk = bulk.drop(columns=['UDI'])
train_df = pd.concat([train, bulk], axis=0)
test_df = pd.read_csv("test.csv", index_col=0)

In [3]:
train_df.head()

,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [4]:
X = train_df.drop(columns=['Machine failure'])
y = train_df['Machine failure']

In [5]:
feature_names = ['Product ID','Type',
                 'Air temperature [K]','Process temperature [K]',
                 'Rotational speed [rpm]','Torque [Nm]',
                 'Tool wear [min]','TWF','HDF',
                 'PWF','OSF','RNF']

train_pool = Pool(X.to_numpy(), y.to_numpy(), 
                  feature_names=feature_names, 
                  cat_features=['Product ID', 'Type'])

In [9]:
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize CatBoostClassifier
# model = CatBoostClassifier(iterations=1500, learning_rate=0.02, depth=7)

# # Create pool
# train_pool = Pool(X_train, y_train)
# valid_pool = Pool(X_valid, y_valid)

# # Conduct cross-validation
# params = model.get_params()
# cv_data = cv(params = params, pool = train_pool, fold_count=5, plot=True)

# # Extract best round
# best_round = np.argmax(cv_data['test-Accuracy-mean'])

# # Fit model with best round
# model.fit(train_pool, eval_set=valid_pool, verbose=False, plot=True, early_stopping_rounds=best_round)

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="M23773": Cannot convert 'b'M23773'' to float

In [10]:
model = CatBoostClassifier(iterations=2000, learning_rate=0.02, depth=8)
model.fit(train_pool, eval_set=train_pool, verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [11]:
train_auc = AUC(train["Machine failure"], model.predict(train.drop(columns=["Machine failure"])))
print(f"Train Score: {train_auc}")

Train Score: 0.9058808374228773


In [12]:
test_pool = Pool(test_df.to_numpy(), 
                 feature_names=feature_names, 
                 cat_features=['Product ID', 'Type'])
pred = model.predict_proba(test_pool)[:, 1]
pred

array([0.00106075, 0.00208216, 0.00034873, ..., 0.00051742, 0.00164063,
       0.00080096])

In [14]:
sub = pd.read_csv('sample_submission.csv')
sub['Machine failure'] = pred
sub.to_csv('submission_cat_4.csv', index=False)